In [1]:
# Import libraries
import torch
import numpy as np
from pytorch_lightning import Trainer

# Import other complementary scripts
from models.pl_inference import ReidPrediction
from datasets.dataloader import ReidDataModule
from utils.cosine_similarity import cosine_similarity
from utils.re_ranking import re_ranking
from utils.evaluation import evaluate
from utils.visualizations import imshow

In [2]:
#Parameteres
class Config():
    cat_testing_dir = 'data/test/images/'
    cat_anno_test_file = 'data/test/test_anno.csv'
    evaluation_file = 'data/test/gt_test_plain.json'
    number_workers = 8
    num_classes = 100
    batch_size_test = 1
    transformation = True
    size_full_image = (256,512)
    embeddings = 2560
    arcface = False
    backbone = 'resnet152'
    deterministic = [True, "warn"]
    precision = "16-mixed"
    base_model = None

# Creating dataloader
cat_data = ReidDataModule(data_directory=Config(),
                            batch_size_test = Config().batch_size_test,
                            transform=Config().transformation,
                            num_workers= Config().number_workers, 
                            size_full_image = Config().size_full_image)
# Call the setup method
cat_data.setup()

# Create the model
eval_model = ReidPrediction(
    backbone_model = Config().backbone,
    number_classes = Config().num_classes, 
    embedding_size = Config().embeddings,
    arcface = False)

# Create the trainer
trainer = Trainer(accelerator='gpu', logger = False, 
                enable_checkpointing=False, 
                precision = Config().precision, 
                deterministic = Config().deterministic)

# Load the weights and biases
eval_model.full_image_model.load_state_dict(torch.load('pretrained_weights/eval_model.pth'))

# Evaluation of model
for batch_idx,batch in enumerate(cat_data.test_dataloader()):

    label = batch[1]
    img_idx = batch[2]

    print(label,img_idx)

    eval_model.eval()
    eval_model.on_predict_start()
    predictions = eval_model.predict_step([batch[0],batch[1],batch[2]],batch_idx)
    eval_model.on_predict_epoch_end()

    break

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


tensor([3]) tensor([25])
